In [1]:
import cv2  # Nhập thư viện OpenCV để xử lý hình ảnh và video
import os   # Nhập thư viện os để tương tác với hệ thống tệp
import time # Nhập thư viện time để làm việc với thời gian

In [ ]:
def generate_dataset():
    # Tạo một đối tượng CascadeClassifier để phát hiện khuôn mặt
    face_classifier = cv2.CascadeClassifier("C:\\Users\\Admin\\Documents\\EMBEDDED_LAB\\FACE_RECOGNITION\\BTL_THCS\\code\\haarcascade_frontalface_default.xml")
    
    # Hàm để cắt khuôn mặt từ hình ảnh
    def face_cropped(img):
        # Chuyển đổi hình ảnh sang màu xám
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Phát hiện khuôn mặt trong hình ảnh
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5
         
        # Nếu không phát hiện khuôn mặt, trả về None
        if faces is ():
            return None
        # Cắt khuôn mặt từ hình ảnh
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
     
    # Mở camera (0 là camera máy tính, 1 là camera ngoài)
    cap = cv2.VideoCapture(0) 
    time.sleep(2)  # Đợi 2 giây để camera khởi động
    id = 0  # ID của người dùng
    img_id = 0  # Biến đếm số lượng ảnh đã chụp
    name = "Trinh_Quoc_Viet"  # Tên của người lưu ảnh
    
    while True:
        # Đọc khung hình từ camera
        ret, frame = cap.read()
        # Nếu phát hiện khuôn mặt trong khung hình
        if face_cropped(frame) is not None:
            img_id += 1  # Tăng biến đếm ảnh
            # Thay đổi kích thước khuôn mặt đã cắt
            face = cv2.resize(face_cropped(frame), (200, 200))
            # Tạo đường dẫn để lưu ảnh
            file_name_path = "C:\\Users\\Admin\\Documents\\EMBEDDED_LAB\\FACE_RECOGNITION\\BTL_THCS\\dataset\\" + name + "." + str(id) + "." + str(img_id) + ".jpg"
            # Lưu ảnh khuôn mặt đã cắt
            cv2.imwrite(file_name_path, face)
            # Hiển thị số lượng ảnh đã chụp lên khuôn mặt
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
             
            # Hiển thị khuôn mặt đã cắt
            cv2.imshow("Cropped face", face)
             
        # Dừng vòng lặp khi nhấn phím Enter hoặc đã chụp đủ 500 ảnh
        if cv2.waitKey(1) == 13 or int(img_id) == 150:  # 13 là mã ASCII của phím Enter
            break
             
    # Giải phóng camera và đóng tất cả các cửa sổ
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")

# Gọi hàm để bắt đầu quá trình tạo tập dữ liệu
generate_dataset()


<>:15: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\Admin\AppData\Local\Temp\ipykernel_5300\1974299625.py:15: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


In [7]:
from PIL import Image  # Nhập thư viện PIL để xử lý hình ảnh
import os              # Nhập thư viện os để tương tác với hệ thống tệp
import re              # Nhập thư viện re để làm việc với biểu thức chính quy

import matplotlib.pyplot as plt  # Nhập thư viện matplotlib để vẽ đồ thị
import numpy as np               # Nhập thư viện numpy để xử lý mảng và tính toán số học

import pylab as pl               # Nhập thư viện pylab để sử dụng các hàm vẽ đồ thị
from matplotlib import pyplot as plt  # Nhập lại pyplot từ matplotlib (có thể không cần thiết)
from sklearn.model_selection import train_test_split  # Nhập hàm để chia dữ liệu thành tập huấn luyện và kiểm tra
from sklearn.model_selection import GridSearchCV       # Nhập hàm để tìm kiếm tham số tối ưu cho mô hình
from sklearn.metrics import classification_report       # Nhập hàm để đánh giá mô hình phân loại
from sklearn.decomposition import PCA                   # Nhập hàm PCA để giảm chiều dữ liệu
from sklearn.svm import SVC                             # Nhập mô hình SVM để phân loại
from skimage.feature import hog                          # Nhập hàm HOG để trích xuất đặc trưng hình ảnh


In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu hình ảnh
folder = 'C:\\Users\\Admin\\Documents\\EMBEDDED_LAB\\FACE_RECOGNITION\\BTL_THCS\\dataset'
X = []  # Danh sách để lưu trữ các đặc trưng hình ảnh
Y = []  # Danh sách để lưu trữ nhãn tương ứng với các hình ảnh

# Từ điển ánh xạ tên người với nhãn số (có thể tùy chỉnh)
label_dict = {
    "Trinh_Quoc_Viet": 1,
    "Le_Trung_Thanh": 3,
    "Dao_Ha_Thai_Son": 4
    # Thêm người mới ở đây, ví dụ: "Ten_Nguoi_Moi": 4
}

# Duyệt qua tất cả các tệp trong thư mục
for filename in os.listdir(folder):
    # Mở hình ảnh từ tệp
    img = Image.open(os.path.join(folder, filename))
    # Thay đổi kích thước hình ảnh về 128x128 pixel
    img = img.resize([128, 128])
    
    # Trích xuất đặc trưng HOG từ hình ảnh
    fd, hog_image = hog(img, 
                      orientations=12,          # Số lượng hướng
                      pixels_per_cell=(4, 4),  # Kích thước ô
                      cells_per_block=(3, 3),   # Kích thước khối
                      visualize=True,           # Hiển thị hình ảnh HOG
                      channel_axis=-1)          # Trục kênh cho hình ảnh màu
    X.append(fd)  # Thêm đặc trưng HOG vào danh sách X
    
    # Gán nhãn dựa trên tên người trong tên file
    label_assigned = False  # Biến để kiểm tra xem nhãn đã được gán hay chưa
    for name, label in label_dict.items():
        # Kiểm tra xem tên trong từ điển có khớp với tên file không
        if re.match(f"{name}*", filename):
            Y.append(label)  # Thêm nhãn vào danh sách Y
            label_assigned = True  # Đánh dấu là đã gán nhãn
            break
    if not label_assigned:
        Y.append(-1)  # Nhãn không xác định nếu không khớp với bất kỳ tên nào



In [9]:
fd.shape  # Lấy kích thước của mảng đặc trưng HOG (fd)
print(fd.shape)  # In kích thước của mảng đặc trưng HOG ra màn hình

(97200,)


In [10]:

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# X_train: tập đặc trưng cho huấn luyện
# X_test: tập đặc trưng cho kiểm tra
# y_train: nhãn tương ứng cho tập huấn luyện
# y_test: nhãn tương ứng cho tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# In ra nhãn của tập huấn luyện
# print(X_train)
print(y_train)  # In ra nhãn của tập huấn luyện

# print(X_test)
print(y_test)  # In ra nhãn của tập kiểm tra


[4, 4, 1, 3, 5, 4, 4, 1, 4, 3, 1, 4, 1, 5, 1, 1, 4, 4, 4, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 3, 5, 3, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 5, 5, 1, 3, 1, 4, 1, 1, 1, 1, 4, 1, 4, 4, 1, 1, 4, 3, 3, 4, 1, 1, 3, 4, 1, 3, 3, 4, 1, 3, 1, 3, 4, 1, 1, 3, 3, 4, 1, 1, 3, 1, 1, 1, 5, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 3, 1, 1, 1, 4, 1, 3, 1, 1, 1, 3, 1, 5, 1, 1, 4, 1, 1, 5, 1, 1, 4, 4, 1, 3, 1, 1, 1, 1, 1, 4, 4, 3, 1, 4, 4, 1, 3, 1, 1, 3, 1, 3, 1, 3, 1, 1, 4, 1, 3, 4, 3, 4, 1, 5, 3, 1, 1, 4, 1, 1, 3, 1, 3, 4, 1, 1, 4, 1, 4, 1, 4, 1, 4, 4, 4, 4, 1, 4, 1, 1, 1, 1, 1, 4, 3, 1, 1, 3, 3, 5, 3, 1, 5, 3, 1, 1, 1, 1, 3, 3, 4, 1, 1, 1, 1, 1, 5, 3, 1, 1, 3, 1, 1, 1, 4, 4, 4, 1, 3, 1, 1, 1, 3, 3, 3, 1, 4, 3, 4, 5, 3, 3, 4, 3, 1, 1, 3, 4, 3, 3, 1, 3, 3, 4, 1, 1, 1, 4, 1, 3, 4, 1, 3, 3, 4, 1, 3, 1, 4, 3, 1, 1, 1, 3, 1, 4, 1, 5, 1, 4, 1, 3, 1, 1, 4, 3, 1, 1, 1, 4, 1, 5, 1, 1, 1, 1, 1, 4, 1, 1, 1, 4, 4, 1, 4, 5, 3, 1, 4, 4, 1, 1, 3, 1, 3, 1, 1, 4, 4, 1, 1, 1, 1, 5, 1, 3, 1, 1, 5, 1, 4, 1, 4, 1, 4, 1, 3, 4, 1, 1, 3, 1, 5, 1, 1, 

In [11]:

n_components = 50  # Số lượng thành phần chính (components) cần giữ lại
# Khởi tạo đối tượng PCA với số lượng thành phần đã chỉ định và bật tính năng làm trắng (whiten)
# Sau đó, thực hiện việc huấn luyện PCA trên dữ liệu huấn luyện
pca = PCA(n_components=n_components, whiten=True).fit(X_train) 
print(pca)  # In ra thông tin về đối tượng PCA đã được huấn luyện

PCA(n_components=50, whiten=True)


In [12]:
# Chiếu dữ liệu huấn luyện vào không gian chiều thấp hơn bằng cách sử dụng PCA
X_train_pca = pca.transform(X_train)

# Chiếu dữ liệu kiểm tra vào không gian chiều thấp hơn bằng cách sử dụng PCA
X_test_pca = pca.transform(X_test)

# In ra dữ liệu huấn luyện đã được chiếu vào không gian chiều thấp hơn
print(X_train_pca)

# In ra dữ liệu kiểm tra đã được chiếu vào không gian chiều thấp hơn
print(X_test_pca)

[[-0.32297041  0.42154224 -0.77493374 ...  0.31419693 -0.47994467
  -1.60142621]
 [-0.43463665  0.36440187 -0.56708416 ... -0.12448785  1.22299023
   1.57509192]
 [ 0.89032386 -1.90124909  0.47062777 ...  0.87818601  0.41754595
  -0.37481885]
 ...
 [ 0.66451628 -1.60963245 -0.06229927 ... -2.74877449 -1.20414485
  -0.77571579]
 [ 1.16038673 -2.24320666  0.34016397 ... -0.33829546 -0.19120031
  -0.97461173]
 [-0.75763036  0.13374489 -1.1927038  ... -2.38319054 -0.69255489
   0.70213095]]
[[-0.53507068  0.01361824  1.51983323 ...  0.01903042  0.37803112
   0.44536532]
 [ 1.11929035 -2.09891609  0.41482415 ... -0.65950471 -0.96532824
  -0.70356095]
 [-0.07964014 -0.16325354 -0.36324456 ... -1.40607923 -0.3833225
   0.91735613]
 ...
 [-0.41952058  0.04995493 -0.8339022  ...  0.3546589  -0.70859981
  -0.69144173]
 [ 2.1562056   1.49995568 -0.10042828 ... -0.06749171 -0.2953823
   0.21803034]
 [-0.49657148 -0.10170192 -1.08622443 ... -0.91543912  0.46183883
   0.88670089]]


In [13]:

import pickle as pk  # Nhập thư viện pickle để lưu trữ và tải các đối tượng Python

# Lưu đối tượng PCA vào tệp "c1_PCA.pkl" bằng cách sử dụng pickle
# "wb" chỉ định chế độ ghi nhị phân
pk.dump(pca, open("c1_PCA.pkl", "wb"))

In [14]:

# In thông báo trạng thái để cho biết quá trình huấn luyện phân loại bắt đầu
print("Fitting the classifier to the training set")

# Định nghĩa lưới tham số cho việc điều chỉnh siêu tham số
param_grid = {
    'C': [1e2, 1e3, 5e3, 1e4, 5e4, 1e5],  # Các giá trị tham số điều chuẩn để thử nghiệm
    'gamma': [0.00001, 0.0001, 0.0005, 0.001, 0.005],  # Hệ số kernel cho kernel 'rbf' và 'poly'
    'kernel': ['rbf', 'poly'],  # Các loại kernel để đánh giá
    'degree': [2, 3],  # Bậc đa thức cho kernel 'poly'
    'class_weight': ['balanced']  # Tự động điều chỉnh trọng số theo tỷ lệ nghịch với tần suất lớp
}

# Khởi tạo GridSearchCV với bộ phân loại SVC
# SVC được cấu hình ban đầu với kernel 'rbf', trọng số lớp cân bằng và bật tính năng ước lượng xác suất
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced', probability=True), param_grid)

# Huấn luyện GridSearchCV trên dữ liệu huấn luyện (đặc trưng đã được biến đổi PCA và nhãn)
clf = clf.fit(X_train_pca, y_train)

# In ra bộ siêu tham số tốt nhất tìm được từ grid search
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set


c:\Users\Admin\Documents\EMBEDDED_LAB\FACE_RECOGNITION\BTL_THCS\face_env\Lib\site-packages\numpy\ma\core.py:2892: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best estimator found by grid search:
SVC(C=100.0, class_weight='balanced', degree=2, gamma=1e-05, probability=True)


In [15]:
# Nhập thư viện joblib để lưu mô hình
import joblib

# Lưu mô hình tốt nhất từ GridSearchCV vào file 'c2_svm_classifier.pkl' với nén dữ liệu (compress=1)
joblib.dump(clf.best_estimator_, 'c2_svm_classifier.pkl', compress=1)  # Chỉ lưu các tham số tốt nhất

['c2_svm_classifier.pkl']

In [16]:
# Dự đoán nhãn cho tập kiểm tra bằng mô hình tốt nhất từ GridSearchCV
y_pred = clf.predict(X_test_pca)

# Hiển thị kết quả dự đoán
y_pred

array([1, 1, 1, 1, 3, 1, 1, 3, 5, 1, 4, 1, 4, 1, 1, 1, 1, 1, 4, 3, 1, 1,
       1, 4, 3, 1, 1, 3, 1, 1, 4, 1, 1, 4, 1, 4, 1, 1, 1, 3, 1, 1, 1, 3,
       4, 1, 5, 3, 1, 1, 3, 1, 4, 1, 4, 1, 1, 1, 1, 1, 5, 4, 3, 5, 1, 1,
       4, 1, 1, 3, 1, 1, 1, 4, 1, 4, 3, 1, 4, 1, 1, 4, 1, 3, 1, 1, 4, 1,
       1, 3, 3, 4, 3, 4, 1, 3, 1, 1, 1, 3, 4, 1, 3, 4, 1, 1, 4, 1, 5, 1,
       1, 3, 1, 1, 1, 1, 4, 1, 1, 4, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1, 4,
       1, 1, 1, 1, 4, 1, 3, 3, 3, 3, 1, 3, 1, 1, 1, 4, 3, 5, 1, 1, 1, 3,
       5, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 1, 4, 3, 3, 1, 4, 1, 1, 1, 3,
       1, 1, 1, 3, 4, 4, 1, 3, 1, 1, 1, 1, 3, 5, 1, 1, 1, 5, 1, 1, 5, 1,
       3, 1, 1, 3, 1, 1, 3, 1, 3, 4, 3, 3, 5, 1, 4, 3, 1, 3, 1, 3, 1, 4,
       1, 1, 1, 1, 3, 3, 4, 1, 1, 1, 1, 4, 1, 4, 1, 4, 3, 4, 1, 4, 4, 1,
       4, 1, 1, 1, 1, 4, 4, 3, 1, 4, 4, 4, 1, 1, 3, 1, 3, 3, 4, 3, 4, 1,
       4, 5, 1, 1, 1, 1, 5, 1, 3, 4, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 3, 5, 4, 4, 1, 4, 1, 1, 1, 1, 3, 3,

In [17]:
# Nhập hàm confusion_matrix từ sklearn.metrics để tính ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix

# Nhập thư viện matplotlib.pyplot để hỗ trợ vẽ biểu đồ (nếu cần)
import matplotlib.pyplot as plt

# In báo cáo phân loại, bao gồm các chỉ số như precision, recall, f1-score cho từng lớp
print(classification_report(y_test, y_pred))

# In ma trận nhầm lẫn để đánh giá số lượng dự đoán đúng và sai theo từng lớp
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       273
           3       1.00      1.00      1.00        90
           4       1.00      1.00      1.00       100
           5       1.00      1.00      1.00        27

    accuracy                           1.00       490
   macro avg       1.00      1.00      1.00       490
weighted avg       1.00      1.00      1.00       490

[[273   0   0   0]
 [  0  90   0   0]
 [  0   0 100   0]
 [  0   0   0  27]]
